# Data Cleansing


In [467]:
MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\Original'
#MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\test'
NEIL_ORIGINALS = r'C:\DS Project\Images\Neil\Original'
MANUALLY_IDENTIFIED_TRACTORS =  r'C:\DS Project\Images\Tractors'

SET_SECONDS = 2

## Imports

In [468]:
import psycopg2
import db_config as creds
import os
import time
import datetime
from stat import * 

In [469]:
class theImg:
  def __init__(self, filename, createdDate,cameraName,cameraId,folderLocation):
    self.createdDate = createdDate
    self.filename = filename
    self.cameraName = cameraName
    self.cameraId = cameraId
    self.folderLocation = folderLocation
        
    # see if the image has previously been identifed as a tractor
    self.MWIdentified = False
    for subdir, dirs, files in os.walk(MANUALLY_IDENTIFIED_TRACTORS):
        for tractorFilename in files:
            if tractorFilename == filename:
                self.MWIdentified = True
    

In [470]:
def processImages(imgLocation, cameraName, cameraId):
    imgSet = 1
    imagesInSet = []

    # loop round each image in the original folder
    for subdir, dirs, files in os.walk(imgLocation):
        
        for filename in files:
        
            filepath = subdir + os.sep + filename
            stat = os.stat(filepath)
        
            # create new image object with attributes
            createdDate = datetime.datetime.fromtimestamp(stat[ST_MTIME])
               
            newImg = theImg(filename,createdDate,cameraName,cameraId,subdir + os.sep)
        
            # check if image is > SET_SECONDS seconds offset the last image
            if (len(imagesInSet) > 0) and ( (createdDate-imagesInSet[len(imagesInSet)-1].createdDate).total_seconds() > SET_SECONDS ):
            
                # insert each of the images in the list into the database
                addSetToDB(imagesInSet)
                
                # increment the set counter
                imgSet+=1
            
                # clear out the list
                imagesInSet.clear()
        
            #set the image set 
            newImg.imgSet = imgSet    
            
            # add the image to the list
            imagesInSet.append(newImg)
        
    # add the last set to the data base     
    addSetToDB(imagesInSet) 

## Database functions

In [471]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)

    # Create a cursor object
    cursor = conn.cursor()
  
    print ("connected!")
    return conn, cursor

def disconnect(conn,cursor):
    
    cursor.close()
    conn.close()
    print ("disconnected!")
    
def addSetToDB(imgSet):
    
    sql = """INSERT INTO images ("fileName", "cameraId", "cameraName", "dateTaken", "setId", "MW_ManuallyIdentifiedTractor", "folderLocation") VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
    # Connecting to DB
    conn, cursor = connect()
    
    for img in imgSet:

        #insert into db
        cursor.execute(sql, (img.filename,img.cameraId,img.cameraName, img.createdDate, img.imgSet, img.MWIdentified, img.folderLocation,))
        conn.commit()
        
        print ("Inserted..",img.filename,img.cameraId,img.cameraName, img.createdDate, img.imgSet, img.MWIdentified, img.folderLocation)
        
    # Disconnect from DB
    disconnect(conn, cursor)

# Process Original Images from Mike and Neil

In [472]:
#processImages(MIKE_ORIGINALS, "Mike", 1) 
processImages(NEIL_ORIGINALS, "Neil", 2) 

connected!
Inserted.. 01PT0001.JPG 2 Neil 2019-08-09 10:29:24 1 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0002.JPG 2 Neil 2019-08-09 10:29:40 2 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0004.JPG 2 Neil 2019-08-09 11:55:10 3 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0005.JPG 2 Neil 2019-08-09 12:34:32 4 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0006.JPG 2 Neil 2019-08-09 13:11:40 5 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0007.JPG 2 Neil 2019-08-09 13:32:32 6 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0008.JPG 2 Neil 2019-08-09 13:48:24 7 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!

Inserted.. 01PT0064.JPG 2 Neil 2019-08-13 08:40:18 58 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0065.JPG 2 Neil 2019-08-13 10:45:58 59 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0066.JPG 2 Neil 2019-08-13 10:46:50 60 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0067.JPG 2 Neil 2019-08-13 10:55:46 61 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0068.JPG 2 Neil 2019-08-13 11:06:56 62 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0069.JPG 2 Neil 2019-08-13 11:15:16 63 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0070.JPG 2 Neil 2019-08-13 11:15:32 64 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
con

connected!
Inserted.. 01PT0122.JPG 2 Neil 2019-08-15 09:30:38 115 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0123.JPG 2 Neil 2019-08-15 09:35:14 116 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0124.JPG 2 Neil 2019-08-15 10:13:22 117 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0126.JPG 2 Neil 2019-08-15 11:17:42 118 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0127.JPG 2 Neil 2019-08-15 12:01:40 119 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0128.JPG 2 Neil 2019-08-15 12:24:14 120 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0129.JPG 2 Neil 2019-08-15 12:28:12 121 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\

connected!
Inserted.. 01PT0181.JPG 2 Neil 2019-08-17 08:10:56 173 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0182.JPG 2 Neil 2019-08-17 09:47:42 174 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0183.JPG 2 Neil 2019-08-17 10:02:06 175 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0184.JPG 2 Neil 2019-08-17 10:05:20 176 True C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0185.JPG 2 Neil 2019-08-17 10:36:08 177 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0187.JPG 2 Neil 2019-08-17 10:53:08 178 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0188.JPG 2 Neil 2019-08-17 13:06:16 179 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\


Inserted.. 01PT0241.JPG 2 Neil 2019-08-20 14:13:52 230 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0242.JPG 2 Neil 2019-08-20 14:23:24 231 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0243.JPG 2 Neil 2019-08-20 15:02:20 232 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0244.JPG 2 Neil 2019-08-20 15:07:28 233 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0245.JPG 2 Neil 2019-08-20 15:21:24 234 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0246.JPG 2 Neil 2019-08-21 08:05:28 235 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnected!
connected!
Inserted.. 01PT0247.JPG 2 Neil 2019-08-21 10:59:22 236 False C:\DS Project\Images\Neil\Original\01 9 Aug - 31 Aug 2019\
disconnect

...

connected!
Inserted.. 23PT3181.JPG 2 Neil 2020-06-22 15:58:54 9977 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3182.JPG 2 Neil 2020-06-22 15:59:12 9978 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3183.JPG 2 Neil 2020-06-22 15:59:32 9979 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3184.JPG 2 Neil 2020-06-22 15:59:52 9980 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3185.JPG 2 Neil 2020-06-22 16:00:10 9981 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3186.JPG 2 Neil 2020-06-22 16:00:28 9982 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3187.JPG 2 Neil 2020-06-22 16:01:20 9983 False C:\DS Project\Images\Neil\Origina

connected!
Inserted.. 23PT3237.JPG 2 Neil 2020-06-22 17:43:54 10033 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3238.JPG 2 Neil 2020-06-22 17:59:22 10034 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3239.JPG 2 Neil 2020-06-22 18:15:02 10035 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3240.JPG 2 Neil 2020-06-22 19:06:02 10036 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3241.JPG 2 Neil 2020-06-22 21:05:22 10037 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3242.JPG 2 Neil 2020-06-22 21:18:20 10038 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3243.JPG 2 Neil 2020-06-23 01:49:54 10039 False C:\DS Project\Images\Neil\

connected!
Inserted.. 23PT3293.JPG 2 Neil 2020-06-23 13:28:34 10089 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3294.JPG 2 Neil 2020-06-23 13:32:36 10090 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3295.JPG 2 Neil 2020-06-23 13:38:48 10091 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3296.JPG 2 Neil 2020-06-23 13:42:32 10092 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3297.JPG 2 Neil 2020-06-23 13:43:32 10093 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3298.JPG 2 Neil 2020-06-23 13:45:44 10094 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3299.JPG 2 Neil 2020-06-23 13:46:08 10095 False C:\DS Project\Images\Neil\

connected!
Inserted.. 23PT3348.JPG 2 Neil 2020-06-23 14:48:12 10144 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3349.JPG 2 Neil 2020-06-23 14:48:54 10145 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3350.JPG 2 Neil 2020-06-23 14:50:12 10146 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3351.JPG 2 Neil 2020-06-23 14:54:06 10147 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3352.JPG 2 Neil 2020-06-23 15:12:36 10148 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3353.JPG 2 Neil 2020-06-23 15:14:48 10149 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3354.JPG 2 Neil 2020-06-23 15:26:54 10150 False C:\DS Project\Images\Neil\

connected!
Inserted.. 23PT3403.JPG 2 Neil 2020-06-24 14:24:04 10199 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3404.JPG 2 Neil 2020-06-24 14:24:36 10200 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3405.JPG 2 Neil 2020-06-24 14:42:54 10201 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3406.JPG 2 Neil 2020-06-24 15:03:42 10202 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3407.JPG 2 Neil 2020-06-24 15:18:10 10203 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3408.JPG 2 Neil 2020-06-24 15:30:14 10204 False C:\DS Project\Images\Neil\Original\23 21 June - 26 June 2020\
disconnected!
connected!
Inserted.. 23PT3409.JPG 2 Neil 2020-06-24 15:44:04 10205 False C:\DS Project\Images\Neil\